<a href="https://colab.research.google.com/github/Rizwankaka/Agentic-AI-/blob/main/Swarm/swarm_greeting_news_search_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Greeting/News Search Agent by Swarm (OpenAI)

In [2]:
%pip install -qU git+https://github.com/openai/swarm.git requests

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


* swarm: A library for creating and managing multiple AI agents.
* requests: A library for making HTTP requests, which we'll use to interact with APIs.

In [3]:
# Set the API keys as environment variables
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['NEWS_API_KEY'] = userdata.get('NEWS_API_KEY')

In [24]:
# Access the API key
news_api_key = os.getenv('NEWS_API_KEY')

In [4]:
# Import required libraries
from swarm import Swarm, Agent
import requests
from IPython.display import display, Markdown

## Agent 1 - Greeting Agent
We will define our first agent, the Greeting Agent, which will greet the user and tell a joke.

In [11]:
# Define the Greeting Agent
greeting_agent = Agent(
    name="Greeting Agent",
    instructions="You are a friendly assistant that greets the user and tells a joke.",
)

* **name**: Identifies the agent.
* **Instructions**: Provides the agent with context on how to respond.

## Agent 2 - News Agent
Next, we define the News Agent, which fetches the latest news headlines.

In [17]:
def fetch_news(country="us", category=None):
    """
    Fetches the top news headlines for a given country and category.

    Args:
        country (str): The country code (default is "us").
        category (str): The news category (optional).

    Returns:
        str: A string containing the top news headlines or an error message.
    """
    base_url = "https://newsapi.org/v2/top-headlines"
    params = {
        "country": country,
        "apiKey": news_api_key
    }

    # Add a category if provided
    if category:
        params["category"] = category

    # Make a GET request to the News API
    response = requests.get(base_url, params=params)
    data = response.json()

    # Check if the request was successful
    if response.status_code == 200 and 'articles' in data:
        # Extract the titles of the first 10 articles
        headlines = [article['title'] for article in data['articles'][:10]]
        return f"Here are the top news for {country.upper()} ({category or 'all categories'}):\n" + "\n".join(headlines)
    else:
        # Return an error message if the request failed
        return f"Sorry, I couldn't fetch the news at the moment: {data.get('message', 'Unknown error')}"

This function uses the News API to fetch the latest headlines based on the specified country and category.

In [18]:
# Define the News Agent
news_agent = Agent(
    name="News Agent",
    instructions="You provide the top news headlines for a given country and category.",
    functions=[fetch_news]
)

* **Functions**: List of functions that the agent can use to perform tasks.

## Orchestrating the agents
We define how the main agent will delegate tasks to specialized agents.

In [19]:
# Define function that transfers the task to the agent
def transfer_to_agent(agent_name):
  agents = {
      "Greeting Agent": greeting_agent,
      "News Agent": news_agent
  }
  return agents[agent_name]

* **Functionality**: Returns the agent instance based on the provided agent name.
* **Usage**: The main agent will use this function to transfer control to the appropriate agent.

In [20]:
# Define the Main AI Agent
main_agent = Agent(
    name = "Main Agent",
    instructions = """
    You are the main assistant.
    BAsed on the user's request, you decide which specialized agent should handle the task.
    - If the user wants a greeting or a joke, transfer to the Greeting Agent.
    - If the user wants to fetch the news, transfer to the News Agent.
    """,
    functions = [transfer_to_agent]
)

* **Instructions**: Guides the main agent on how to delegate tasks.
* **Functions**: Includes the transfer_to_agent function to enable delegation.

## Start the Connection to the OpenAI API

We initialize the Swarm client, which handles communication with the OpenAI API and manages the agents.

In [21]:
# Start the connection the OpenAI API
client = Swarm()

We simulate a user interaction with the main agent.

In [12]:
# Run the application
response = client.run(
    agent = main_agent,
    messages = [{"role": "user",
                 "content": "good morning"}]
)
print(display(Markdown(response.messages[-1]['content'])))

Good morning! How are you today? Would you like to hear a joke to start your day with a smile?

None


* **Agent**: We specify the main_agent as the agent to handle the interaction.
* **Messages**: We pass a list of messages, starting with the user's input "good morning".
* **Response Handling**: We display the assistant's response in Markdown format for better readability.

In [13]:
response.agent

Agent(name='Greeting Agent', model='gpt-4o', instructions='You are a friendly assistant that greets the user and tells a joke.', functions=[], tool_choice=None, parallel_tool_calls=True)

In [22]:
# Run the application
response = client.run(
    agent = main_agent,
    messages = [{"role": "user",
                 "content": "what is the current news in USA"}]
)
print(display(Markdown(response.messages[-1]['content'])))

Here are some of the current top news headlines in the USA:

1. "UK’s Starmer in Kyiv for security talks with a pledge for a ‘100-year partnership’ with Ukraine" - The Associated Press.
2. "Netanyahu says Cabinet won't meet over ceasefire until Hamas backs down from 'last minute crisis'" - The Associated Press.
3. "Israel's cabinet set to approve ceasefire deal as at least 20 killed in strikes on Gaza" - BBC.com.
4. "Jeff Bezos' Blue Origin launches massive New Glenn rocket into orbit on 1st flight (video)" - Space.com.
5. "Bollywood Star Saif Ali Khan Stabbed Multiple Times in Mumbai Home Invasion Attack" - Variety.
6. "‘One of Them Days’ Review: Keke Palmer and SZA Make a Rowdily Funny Duo in Issa Rae-Produced Comedy" - Hollywood Reporter.
7. "Samsung Galaxy S25 leak shows new Gemini overlay, hotword, and more" - 9to5Google.
8. "Ukraine hawk removed from House intelligence role, reportedly at Trump’s request" - The Washington Post.

None


In [23]:
response.agent

Agent(name='News Agent', model='gpt-4o', instructions='You provide the top news headlines for a given country and category.', functions=[<function fetch_news at 0x7c56393689a0>], tool_choice=None, parallel_tool_calls=True)

This line outputs the agent that provided the final response, which helps us verify which specialized agent handled the request.